In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [5]:
feature_name_df = pd.read_csv('human_activity/features.txt',
            sep='\s+',
            header=None,
            names=['column_index','column_name'])

In [9]:
feature_dup_df = feature_name_df.groupby('column_name').count()
feature_dup_df.head(1)

,column_index
column_name,
"angle(X,gravityMean)",1


In [11]:
feature_dup_df[feature_dup_df['column_index']>1].count()

column_index    42
dtype: int64

In [12]:
feature_dup_df[feature_dup_df['column_index']>1].head(2)

,column_index
column_name,
"fBodyAcc-bandsEnergy()-1,16",3
"fBodyAcc-bandsEnergy()-1,24",3


In [23]:
def get_new_feature_name_df(old):
    df = pd.DataFrame(data=old.groupby('column_name').cumcount(),
                 columns=['dup_cnt'])
    df = df.reset_index()
    new_df = pd.merge(old.reset_index(),df,how='outer')
    new_df['column_name'] = new_df[['column_name','dup_cnt']].apply(lambda x: x[0]+'_'+str(x[1]) 
                                            if x[1] > 0 
                                            else x[0] ,axis=1)
    new_df.drop(columns=['index'],inplace=True)
    return new_df

In [24]:
df1 = get_new_feature_name_df(feature_name_df)
df1[df1['dup_cnt']>0]

,column_index,column_name,dup_cnt
316,317,"fBodyAcc-bandsEnergy()-1,8_1",1
317,318,"fBodyAcc-bandsEnergy()-9,16_1",1
318,319,"fBodyAcc-bandsEnergy()-17,24_1",1
319,320,"fBodyAcc-bandsEnergy()-25,32_1",1
320,321,"fBodyAcc-bandsEnergy()-33,40_1",1
...,...,...,...
497,498,"fBodyGyro-bandsEnergy()-17,32_2",2
498,499,"fBodyGyro-bandsEnergy()-33,48_2",2
499,500,"fBodyGyro-bandsEnergy()-49,64_2",2
500,501,"fBodyGyro-bandsEnergy()-1,24_2",2


In [20]:
feature_name_df.head(1)

,column_index,column_name
0,1,tBodyAcc-mean()-X
